In [1]:
import pandas as pd
import time
from sklearn.metrics import classification_report
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from pathlib import Path
import time
from tqdm import tqdm

In [2]:
# Settings
train_dir = Path('./data/train')
val_dir   = Path('./data/test')
img_size = 48
batch_size = 64

# Transforms
train_transforms = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(5),
    transforms.RandomAffine(degrees=0, shear=10, translate=(0.2, 0.2)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)  # normalize to [-1, 1]
])

val_transforms = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)
])

# Datasets
train_dataset = datasets.ImageFolder(root=train_dir, transform=train_transforms)
val_dataset   = datasets.ImageFolder(root=val_dir, transform=val_transforms)
print(f"Train images: {len(train_dataset)}")
print(f"Val images: {len(val_dataset)}")

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader   = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

# Class names
class_names = train_dataset.classes
print("Classes:", class_names)

Train images: 33595
Val images: 7178
Classes: ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']


In [3]:
class InvertedResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride, expand_ratio):
        super().__init__()
        self.use_res_connect = stride == 1 and in_channels == out_channels
        hidden_dim = in_channels * expand_ratio

        layers = []

        # Expansion
        if expand_ratio != 1:
            layers.extend([
                nn.Conv2d(in_channels, hidden_dim, kernel_size=1, bias=False),
                nn.BatchNorm2d(hidden_dim),
                nn.ReLU6(inplace=True)
            ])

        # Depthwise
        layers.extend([
            nn.Conv2d(hidden_dim, hidden_dim, kernel_size=3, stride=stride, padding=1, groups=hidden_dim, bias=False),
            nn.BatchNorm2d(hidden_dim),
            nn.ReLU6(inplace=True)
        ])

        # Projection
        layers.extend([
            nn.Conv2d(hidden_dim, out_channels, kernel_size=1, bias=False),
            nn.BatchNorm2d(out_channels)
        ])

        self.block = nn.Sequential(*layers)

    def forward(self, x):
        if self.use_res_connect:
            return x + self.block(x)
        else:
            return self.block(x)

class MobileNetV2_48x48(nn.Module):
    def __init__(self, num_classes=7, dropout_rate=0.0):
        super().__init__()
        self.dropout_rate = dropout_rate
        self.dropout = nn.Dropout(dropout_rate)

        # Initial convolution layer
        self.init_conv = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1, bias=False),  # 48x48 input
            nn.BatchNorm2d(32),
            nn.ReLU6(inplace=True)
        )

        # Bottleneck configuration: (t, c, n, s)
        bottlenecks_cfg = [
            (1, 16, 1, 1),
            (6, 24, 1, 1),  # no downsampling yet
            (6, 32, 2, 2),
            (6, 64, 1, 2),
            (6, 96, 1, 1),
        ]

        blocks = []
        in_channels = 32
        for t, out_channels, n, s in bottlenecks_cfg:
            for i in range(n):
                stride = s if i == 0 else 1
                blocks.append(InvertedResidualBlock(in_channels, out_channels, stride, t))
                in_channels = out_channels
        self.bottlenecks = nn.Sequential(*blocks)

        self.last_conv = nn.Sequential(
            nn.Conv2d(in_channels, 128, kernel_size=1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU6(inplace=True),
            nn.Dropout(dropout_rate)
        )

        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.init_conv(x)
        x = self.bottlenecks(x)
        x = self.last_conv(x)
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        return self.classifier(x)

In [4]:
# Define MobileNetV2 model
mobilenet_model = MobileNetV2_48x48(num_classes=len(class_names))
total_params = sum(p.numel() for p in mobilenet_model.parameters())
print(f"Total parameters: {total_params}")

Total parameters: 132935


In [5]:
if torch.backends.mps.is_available() and torch.backends.mps.is_built():
    device = torch.device("mps")
else:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Using device: {device}")
mobilenet_model = MobileNetV2_48x48(num_classes=len(class_names)).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(mobilenet_model.parameters(), lr=1e-3,  weight_decay=1e-4)

Using device: mps


In [6]:
def train(model, train_loader, val_loader, optimizer, criterion, num_epochs=15, model_name='mobv2_model.pth'):
    history = []  # collect metrics per epoch

    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch+1}/{num_epochs}")
        start_time = time.time()

        # Training phase
        model.train()
        train_loss, correct, total = 0, 0, 0
        for images, labels in tqdm(train_loader, desc="Training"):
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * images.size(0)
            _, predicted = outputs.max(1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

        train_acc = correct / total
        train_loss /= total

        # Validation phase
        model.eval()
        val_loss, correct, total = 0, 0, 0
        y_true, y_pred = [], []

        with torch.no_grad():
            for images, labels in tqdm(val_loader, desc="Validation"):
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * images.size(0)

                _, predicted = outputs.max(1)
                correct += (predicted == labels).sum().item()
                total += labels.size(0)

                y_true.extend(labels.cpu().numpy())
                y_pred.extend(predicted.cpu().numpy())

        val_acc = correct / total
        val_loss /= total

        # Classification report
        report = classification_report(y_true, y_pred, target_names=class_names, output_dict=True)
        val_f1_macro = report['macro avg']['f1-score']

        epoch_time = time.time() - start_time

        # Collect metrics
        epoch_data = {
            'epoch': epoch + 1,
            'train_loss': train_loss,
            'val_loss': val_loss,
            'train_acc': train_acc,
            'val_acc': val_acc,
            'val_f1_macro': val_f1_macro,
            'epoch_time_sec': epoch_time
        }

        # Add per-class metrics
        for cls in class_names:
            for metric in ['precision', 'recall', 'f1-score']:
                key = f'{cls}_{metric}'
                epoch_data[key] = report[cls][metric]

        history.append(epoch_data)

        # Console output
        print("\nValidation Report:")
        for cls in class_names:
            cls_metrics = report[cls]
            print(f"{cls}: Prec={cls_metrics['precision']:.3f} | Rec={cls_metrics['recall']:.3f} | F1={cls_metrics['f1-score']:.3f}")

        print(f"\nEpoch Summary: Train Loss={train_loss:.4f} | Val Loss={val_loss:.4f} | Train Acc={train_acc:.3f} | Val Acc={val_acc:.3f}")
        print(f"Epoch time: {epoch_time:.2f} seconds")

    torch.save(model.state_dict(), f"results/{model_name}")
    print(f"Saved model to 'results/{model_name}'")

    return history

In [ ]:
def build_model(dropout):
    return MobileNetV2_48x48(num_classes=len(class_names), dropout_rate=dropout).to(device)

# Define settings to test
configs = [
    # No dropout, no weight decay
    {'name': 'adam_lr3_drop0_wd0_bn', 'optimizer': 'adam', 'lr': 1e-3, 'dropout': 0.0, 'weight_decay': 0.0},
    # Dropout only
    {'name': 'adam_lr3_drop15_wd0_bn', 'optimizer': 'adam', 'lr': 1e-3, 'dropout': 0.15, 'weight_decay': 0.0},
    # Weight decay only
    {'name': 'adam_lr3_drop0_wd1e4_bn', 'optimizer': 'adam', 'lr': 1e-3, 'dropout': 0.0, 'weight_decay': 1e-4},
    # Combined dropout + weight decay
    {'name': 'adam_lr3_drop15_wd1e4_bn', 'optimizer': 'adam', 'lr': 1e-3, 'dropout': 0.15, 'weight_decay': 1e-4},
    # Same but using SGD to test optimizer difference
    {'name': 'sgd_lr3_drop15_wd1e4_bn', 'optimizer': 'sgd', 'lr': 1e-3, 'dropout': 0.15, 'weight_decay': 1e-4},
]

for config in configs:
    print(f"\nTraining config: {config['name']}")
    model = build_model(config['dropout'])

    if config['optimizer'] == 'adam':
        optimizer = optim.Adam(model.parameters(), lr=config['lr'], weight_decay=config['weight_decay'])
    else:
        optimizer = optim.SGD(model.parameters(), lr=config['lr'], momentum=0.9, weight_decay=config['weight_decay'])

    criterion = nn.CrossEntropyLoss()
    history = train(model, train_loader, val_loader, optimizer, criterion, num_epochs=15, model_name=config['name'])
    pd.DataFrame(history).to_csv(f'results/history_mobv2_{config["name"]}.csv', index=False)


Training config: adam_lr3_drop0_wd0_bn

Epoch 1/15


Validation: 100%|██████████| 113/113 [00:19<00:00,  5.73it/s]



Validation Report:
angry: Prec=0.400 | Rec=0.006 | F1=0.012
disgust: Prec=0.015 | Rec=0.207 | F1=0.028
fear: Prec=0.166 | Rec=0.047 | F1=0.073
happy: Prec=0.460 | Rec=0.524 | F1=0.490
neutral: Prec=0.311 | Rec=0.102 | F1=0.154
sad: Prec=0.302 | Rec=0.241 | F1=0.268
surprise: Prec=0.326 | Rec=0.745 | F1=0.454

Epoch Summary: Train Loss=1.6911 | Val Loss=1.8743 | Train Acc=0.335 | Val Acc=0.286
Epoch time: 148.02 seconds

Epoch 2/15


Validation: 100%|██████████| 113/113 [00:19<00:00,  5.78it/s]



Validation Report:
angry: Prec=0.328 | Rec=0.043 | F1=0.076
disgust: Prec=0.019 | Rec=0.450 | F1=0.037
fear: Prec=0.230 | Rec=0.044 | F1=0.074
happy: Prec=0.739 | Rec=0.422 | F1=0.537
neutral: Prec=0.448 | Rec=0.251 | F1=0.322
sad: Prec=0.322 | Rec=0.382 | F1=0.349
surprise: Prec=0.523 | Rec=0.673 | F1=0.588

Epoch Summary: Train Loss=1.4444 | Val Loss=2.0699 | Train Acc=0.447 | Val Acc=0.311
Epoch time: 149.98 seconds

Epoch 3/15


...


Validation Report:
angry: Prec=0.253 | Rec=0.044 | F1=0.075
disgust: Prec=0.017 | Rec=0.279 | F1=0.032
fear: Prec=0.261 | Rec=0.062 | F1=0.100
happy: Prec=0.533 | Rec=0.528 | F1=0.530
neutral: Prec=0.337 | Rec=0.132 | F1=0.190
sad: Prec=0.335 | Rec=0.161 | F1=0.218
surprise: Prec=0.318 | Rec=0.813 | F1=0.457

Epoch Summary: Train Loss=1.4406 | Val Loss=2.0510 | Train Acc=0.446 | Val Acc=0.294
Epoch time: 158.27 seconds

Epoch 14/15


Validation: 100%|██████████| 113/113 [00:19<00:00,  5.87it/s]



Validation Report:
angry: Prec=0.224 | Rec=0.011 | F1=0.022
disgust: Prec=0.019 | Rec=0.342 | F1=0.036
fear: Prec=0.181 | Rec=0.031 | F1=0.053
happy: Prec=0.464 | Rec=0.626 | F1=0.533
neutral: Prec=0.391 | Rec=0.096 | F1=0.154
sad: Prec=0.327 | Rec=0.125 | F1=0.181
surprise: Prec=0.362 | Rec=0.767 | F1=0.491

Epoch Summary: Train Loss=1.4220 | Val Loss=2.0804 | Train Acc=0.452 | Val Acc=0.293
Epoch time: 163.49 seconds

Epoch 15/15


Validation: 100%|██████████| 113/113 [00:19<00:00,  5.81it/s]


Validation Report:
angry: Prec=0.237 | Rec=0.105 | F1=0.146
disgust: Prec=0.014 | Rec=0.207 | F1=0.027
fear: Prec=0.201 | Rec=0.053 | F1=0.084
happy: Prec=0.595 | Rec=0.542 | F1=0.567
neutral: Prec=0.383 | Rec=0.147 | F1=0.212
sad: Prec=0.345 | Rec=0.261 | F1=0.297
surprise: Prec=0.361 | Rec=0.795 | F1=0.496

Epoch Summary: Train Loss=1.4050 | Val Loss=1.9028 | Train Acc=0.462 | Val Acc=0.321
Epoch time: 157.97 seconds
Saved model to 'results/sgd_lr3_drop15_wd1e4_bn'


In [12]:
from torch.optim.lr_scheduler import ReduceLROnPlateau
import copy

def train(model, train_loader, val_loader, optimizer, criterion, num_epochs=15, model_name='mobv2_model.pth', use_scheduler=False, use_early_stopping=False, patience=5):
    history = []
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3) if use_scheduler else None

    best_val_loss = float('inf')
    best_model_wts = copy.deepcopy(model.state_dict())
    epochs_no_improve = 0

    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch+1}/{num_epochs}")
        start_time = time.time()

        # Training
        model.train()
        train_loss, correct, total = 0, 0, 0
        for images, labels in tqdm(train_loader, desc="Training"):
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * images.size(0)
            _, predicted = outputs.max(1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

        train_acc = correct / total
        train_loss /= total

        # Validation
        model.eval()
        val_loss, correct, total = 0, 0, 0
        y_true, y_pred = [], []

        with torch.no_grad():
            for images, labels in tqdm(val_loader, desc="Validation"):
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * images.size(0)

                _, predicted = outputs.max(1)
                correct += (predicted == labels).sum().item()
                total += labels.size(0)
                y_true.extend(labels.cpu().numpy())
                y_pred.extend(predicted.cpu().numpy())

        val_acc = correct / total
        val_loss /= total

        # Classification metrics
        report = classification_report(y_true, y_pred, target_names=class_names, output_dict=True)
        val_f1_macro = report['macro avg']['f1-score']
        epoch_time = time.time() - start_time

        # Step scheduler
        if scheduler:
            scheduler.step(val_loss)

        # Check if this is the best model
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model_wts = copy.deepcopy(model.state_dict())
            torch.save(best_model_wts, f"results/best_{model_name}")  # Save best checkpoint
            print("New best model saved.")
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1
            if use_early_stopping and epochs_no_improve >= patience:
                print(f"Early stopping triggered after {patience} epochs with no improvement.")
                break

        # Record metrics
        epoch_data = {
            'epoch': epoch + 1,
            'train_loss': train_loss,
            'val_loss': val_loss,
            'train_acc': train_acc,
            'val_acc': val_acc,
            'val_f1_macro': val_f1_macro,
            'epoch_time_sec': epoch_time
        }
        for cls in class_names:
            for metric in ['precision', 'recall', 'f1-score']:
                epoch_data[f'{cls}_{metric}'] = report[cls][metric]

        history.append(epoch_data)

        # Print summary
        print("\nValidation Report:")
        for cls in class_names:
            cls_metrics = report[cls]
            print(f"{cls}: Prec={cls_metrics['precision']:.3f} | Rec={cls_metrics['recall']:.3f} | F1={cls_metrics['f1-score']:.3f}")
        print(f"\nEpoch Summary: Train Loss={train_loss:.4f} | Val Loss={val_loss:.4f} | Train Acc={train_acc:.3f} | Val Acc={val_acc:.3f}")
        print(f"Epoch time: {epoch_time:.2f} seconds")

    # Save final model
    torch.save(model.state_dict(), f"results/{model_name}")
    print(f"Saved final model to 'results/{model_name}'")

    # Load best model weights
    model.load_state_dict(best_model_wts)

    return history

In [13]:
# Train best model for 40 epochs with learning rate scheduler
best_dropout = 0.15
best_lr = 1e-3
best_weight_decay = 1e-4
best_optimizer_name = 'adam'

model = build_model(best_dropout)

if best_optimizer_name == 'adam':
    optimizer = optim.Adam(model.parameters(), lr=best_lr, weight_decay=best_weight_decay)
else:
    optimizer = optim.SGD(model.parameters(), lr=best_lr, momentum=0.9, weight_decay=best_weight_decay)

criterion = nn.CrossEntropyLoss()

history = train(
    model, 
    train_loader, 
    val_loader, 
    optimizer, 
    criterion, 
    num_epochs=40,
    model_name='mobv2_final_model.pth',
    use_scheduler=True,
    use_early_stopping=True,
    patience=6
)

pd.DataFrame(history).to_csv("results/history_mobv2_final_model.csv", index=False)


Epoch 1/40


Validation: 100%|██████████| 113/113 [00:18<00:00,  5.95it/s]


New best model saved.

Validation Report:
angry: Prec=0.333 | Rec=0.002 | F1=0.004
disgust: Prec=0.013 | Rec=0.324 | F1=0.026
fear: Prec=0.190 | Rec=0.008 | F1=0.015
happy: Prec=0.402 | Rec=0.595 | F1=0.480
neutral: Prec=0.350 | Rec=0.006 | F1=0.011
sad: Prec=0.319 | Rec=0.149 | F1=0.203
surprise: Prec=0.466 | Rec=0.677 | F1=0.552

Epoch Summary: Train Loss=1.6816 | Val Loss=2.1629 | Train Acc=0.338 | Val Acc=0.259
Epoch time: 145.25 seconds

Epoch 2/40


Validation: 100%|██████████| 113/113 [00:19<00:00,  5.84it/s]


New best model saved.

Validation Report:
angry: Prec=0.312 | Rec=0.005 | F1=0.010
disgust: Prec=0.023 | Rec=0.324 | F1=0.043
fear: Prec=0.231 | Rec=0.032 | F1=0.057
happy: Prec=0.596 | Rec=0.657 | F1=0.625
neutral: Prec=0.341 | Rec=0.457 | F1=0.391
sad: Prec=0.360 | Rec=0.237 | F1=0.285
surprise: Prec=0.543 | Rec=0.680 | F1=0.604

Epoch Summary: Train Loss=1.4172 | Val Loss=1.7687 | Train Acc=0.454 | Val Acc=0.371
Epoch time: 142.89 seconds

Epoch 3/40


...


Validation Report:
angry: Prec=0.516 | Rec=0.098 | F1=0.165
disgust: Prec=0.032 | Rec=0.802 | F1=0.061
fear: Prec=0.316 | Rec=0.133 | F1=0.187
happy: Prec=0.913 | Rec=0.457 | F1=0.609
neutral: Prec=0.546 | Rec=0.223 | F1=0.317
sad: Prec=0.536 | Rec=0.107 | F1=0.178
surprise: Prec=0.337 | Rec=0.869 | F1=0.486

Epoch Summary: Train Loss=1.0240 | Val Loss=2.5681 | Train Acc=0.615 | Val Acc=0.315
Epoch time: 156.18 seconds

Epoch 15/40


Validation: 100%|██████████| 113/113 [00:19<00:00,  5.72it/s]



Validation Report:
angry: Prec=0.459 | Rec=0.404 | F1=0.430
disgust: Prec=0.066 | Rec=0.658 | F1=0.121
fear: Prec=0.408 | Rec=0.211 | F1=0.278
happy: Prec=0.860 | Rec=0.670 | F1=0.753
neutral: Prec=0.586 | Rec=0.410 | F1=0.482
sad: Prec=0.507 | Rec=0.263 | F1=0.346
surprise: Prec=0.400 | Rec=0.872 | F1=0.548

Epoch Summary: Train Loss=0.9808 | Val Loss=1.5267 | Train Acc=0.632 | Val Acc=0.477
Epoch time: 169.50 seconds

Epoch 16/40


Validation: 100%|██████████| 113/113 [00:19<00:00,  5.80it/s]

Early stopping triggered after 6 epochs with no improvement.
Saved final model to 'results/mobv2_final_model.pth'
